In [ ]:
# Install all necessary libraries
!pip install nltk spacy pandas sentencepiece langdetect beautifulsoup4 pyspellchecker
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Import libraries
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from langdetect import detect, DetectorFactory
from bs4 import BeautifulSoup
from spellchecker import SpellChecker

# Set random seed for consistent language detection results
DetectorFactory.seed = 0
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

file_path = '/content/drive/My Drive/eng_fre.csv'

# Try reading the file using 'latin-1' encoding
fre_df = pd.read_csv(file_path, encoding='latin-1')

In [ ]:

print("\nEnglish-French Dataset:")
print(fre_df.head())


English-French Dataset:
   Unnamed: 0          English                     French
0        7715  I am a shy boy.  Je suis un garçon timide.
1        7721  I am in a spot.    Je suis dans le pétrin.
2        7722  I am in a spot.   Je suis dans un endroit.
3        7884  I had to do it.     Il m'a fallu le faire.
4        8136  I saw it on TV.      Je l'ai vu à la télé.


In [ ]:
import nltk

# Re-download 'punkt' package
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
# Remove the existing punkt data, if any
nltk.data.path = ['/root/nltk_data']
nltk.download('punkt', force=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk

# Download 'punkt_tab' resource
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:

# Define a text normalization function for English
def normalize_text_english(text):
    if not isinstance(text, str):  # Check if the text is a string
        return ""  # If not, return an empty string
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    tokens = word_tokenize(text)  # Tokenize the text
    return ' '.join(tokens)

# Define a text normalization function for French (retain apostrophes)
def normalize_text_french(text):
    if not isinstance(text, str):  # Check if the text is a string
        return ""  # If not, return an empty string
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s\']', '', text)  # Remove punctuation except apostrophes
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    tokens = word_tokenize(text)  # Tokenize the text
    return ' '.join(tokens)

# Apply normalization to both datasets
fre_df['English_normalized'] = fre_df['English'].fillna('').apply(normalize_text_english)
fre_df['French_normalized'] = fre_df['French'].fillna('').apply(normalize_text_french)

# Display the first few rows to verify normalization
print("\nEnglish-French Dataset after normalization:")
print(fre_df[['English', 'English_normalized', 'French', 'French_normalized']].head())



English-French Dataset after normalization:
           English English_normalized                     French  \
0  I am a shy boy.     i am a shy boy  Je suis un garçon timide.   
1  I am in a spot.     i am in a spot    Je suis dans le pétrin.   
2  I am in a spot.     i am in a spot   Je suis dans un endroit.   
3  I had to do it.     i had to do it     Il m'a fallu le faire.   
4  I saw it on TV.     i saw it on tv      Je l'ai vu à la télé.   

          French_normalized  
0  je suis un garçon timide  
1    je suis dans le pétrin  
2   je suis dans un endroit  
3   il m ' a fallu le faire  
4      je l'ai vu à la télé  


In [ ]:
# Drop duplicates in both datasets
fre_df.drop_duplicates(subset=['English_normalized', 'French_normalized'], inplace=True)

print("\nDuplicates removed. Sample English-French Dataset:")
print(fre_df[['English_normalized', 'French_normalized']].head())



Duplicates removed. Sample English-French Dataset:
  English_normalized         French_normalized
0     i am a shy boy  je suis un garçon timide
1     i am in a spot    je suis dans le pétrin
2     i am in a spot   je suis dans un endroit
3     i had to do it   il m ' a fallu le faire
4     i saw it on tv      je l'ai vu à la télé


In [ ]:
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Define a function to filter by sentence length
def filter_by_length(text, min_length=5, max_length=50):
    length = len(text.split())
    return min_length <= length <= max_length

# Apply length filtering
fre_df = fre_df[fre_df['English_normalized'].apply(filter_by_length) & fre_df['French_normalized'].apply(filter_by_length)]

print("\nEnglish-French Dataset after length filtering:")
print(fre_df[['English_normalized', 'French_normalized']].head())



English-French Dataset after length filtering:
  English_normalized         French_normalized
0     i am a shy boy  je suis un garçon timide
1     i am in a spot    je suis dans le pétrin
2     i am in a spot   je suis dans un endroit
3     i had to do it   il m ' a fallu le faire
4     i saw it on tv      je l'ai vu à la télé


In [ ]:
!pip install pyspellchecker

In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker()

# Function to correct spelling in English tokens
def correct_spelling(text):
    corrected_text = " ".join([spell.correction(word) if word in spell else word for word in text.split()])
    return corrected_text

# Apply spelling correction on English columns
fre_df['English_normalized'] = fre_df['English_normalized'].apply(correct_spelling)

# Display the datasets after spelling correction
print("\nEnglish-French Dataset after spelling correction:")
print(fre_df[['English_normalized', 'French_normalized']].head())



English-French Dataset after spelling correction:
  English_normalized         French_normalized
0     i am a shy boy  je suis un garçon timide
1     i am in a spot    je suis dans le pétrin
2     i am in a spot   je suis dans un endroit
3     i had to do it   il m ' a fallu le faire
4     i saw it on tv      je l'ai vu à la télé


In [ ]:
from bs4 import BeautifulSoup

# Function to remove HTML tags
def remove_html_tags(text):
    return BeautifulSoup(text, "html.parser").get_text()

# Apply HTML cleaning
fre_df['English_normalized'] = fre_df['English_normalized'].apply(remove_html_tags)
fre_df['French_normalized'] = fre_df['French_normalized'].apply(remove_html_tags)

print("\nEnglish-French Dataset after HTML tag removal:")
print(fre_df[['English_normalized', 'French_normalized']].head())



English-French Dataset after HTML tag removal:
  English_normalized         French_normalized
0     i am a shy boy  je suis un garçon timide
1     i am in a spot    je suis dans le pétrin
2     i am in a spot   je suis dans un endroit
3     i had to do it   il m ' a fallu le faire
4     i saw it on tv      je l'ai vu à la télé


In [ ]:
# Function to check if text is lowercased, punctuation removed, and tokenized
def check_preprocessing(text):
    if not isinstance(text, str):  # Check if input is a string
        return False, False, False  # Return false for all checks if not a string
    lowercased = text == text.lower()  # Check if it's lowercased
    punctuation_removed = not bool(re.search(r'[^\w\s]', text))  # Check if punctuation is removed
    tokenized = len(word_tokenize(text)) > 1  # Check if the text is tokenized (more than one token)
    return lowercased, punctuation_removed, tokenized

# Function to check preprocessing for a specific column in the dataset
def check_data_preprocessing(df, lang_col_name):
    results = df[lang_col_name].fillna('').apply(check_preprocessing)  # Handle NaN values with empty strings
    return results

# Example usage for the English-French dataset
fre_check = check_data_preprocessing(fre_df, 'English')

# Display the preprocessing checks for the English column
print("\nEnglish-French Dataset Preprocessing Check:")
print(fre_check.head())

# Detailed breakdown of preprocessing checks
print("\nDetailed Preprocessing Checks for English-French:")
print(fre_check.apply(lambda x: f"Lowercased: {x[0]}, Punctuation Removed: {x[1]}, Tokenized: {x[2]}").head())



English-French Dataset Preprocessing Check:
0    (False, False, True)
1    (False, False, True)
2    (False, False, True)
3    (False, False, True)
4    (False, False, True)
Name: English, dtype: object

Detailed Preprocessing Checks for English-French:
0    Lowercased: False, Punctuation Removed: False,...
1    Lowercased: False, Punctuation Removed: False,...
2    Lowercased: False, Punctuation Removed: False,...
3    Lowercased: False, Punctuation Removed: False,...
4    Lowercased: False, Punctuation Removed: False,...
Name: English, dtype: object


In [ ]:
# Define a text normalization function for English
def normalize_text_english(text):
    if not isinstance(text, str):  # Check if the text is a string
        return ""  # If not, return an empty string
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    tokens = word_tokenize(text)  # Tokenize the text
    return ' '.join(tokens)

# Define a text normalization function for French (retain apostrophes)
def normalize_text_french(text):
    if not isinstance(text, str):  # Check if the text is a string
        return ""  # If not, return an empty string
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s\']', '', text)  # Remove punctuation except apostrophes
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    tokens = word_tokenize(text)  # Tokenize the text
    return ' '.join(tokens)

# Apply normalization to the entire dataset
fre_df['English_normalized'] = fre_df['English'].fillna('').apply(normalize_text_english)
fre_df['French_normalized'] = fre_df['French'].fillna('').apply(normalize_text_french)

# Verify the preprocessing of the entire dataset
print("\nEnglish-French Dataset after normalization:")
print(fre_df[['English', 'English_normalized', 'French', 'French_normalized']].head())

# Optional: Save the normalized dataset to a CSV file for inspection
output_file = "english_french_dataset_normalized.csv"
fre_df.to_csv(output_file, index=False)
print(f"\nNormalized dataset saved to '{output_file}'.")



English-French Dataset after normalization:
           English English_normalized                     French  \
0  I am a shy boy.     i am a shy boy  Je suis un garçon timide.   
1  I am in a spot.     i am in a spot    Je suis dans le pétrin.   
2  I am in a spot.     i am in a spot   Je suis dans un endroit.   
3  I had to do it.     i had to do it     Il m'a fallu le faire.   
4  I saw it on TV.     i saw it on tv      Je l'ai vu à la télé.   

          French_normalized  
0  je suis un garçon timide  
1    je suis dans le pétrin  
2   je suis dans un endroit  
3   il m ' a fallu le faire  
4      je l'ai vu à la télé  

Normalized dataset saved to 'english_french_dataset_normalized.csv'.


In [ ]:
# Define a normalization function that removes punctuation and tokenizes text
def normalize_text_with_punctuation_removal(text):
    if not isinstance(text, str):  # Check if input is a string
        return ""
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text  # Return normalized text

# Preprocessing check function
def check_preprocessing(text):
    normalized = normalize_text_with_punctuation_removal(text)
    lowercased = normalized == normalized.lower()  # Check if text is lowercase
    punctuation_removed = not bool(re.search(r'[^\w\s]', normalized))  # Check if punctuation is removed
    tokenized = len(word_tokenize(normalized)) > 0  # Check if tokenization is valid
    return lowercased, punctuation_removed, tokenized

# Apply normalization and recheck the preprocessing
fre_df['English_normalized'] = fre_df['English'].fillna('').apply(normalize_text_with_punctuation_removal)
fre_check = fre_df['English_normalized'].apply(check_preprocessing)

# Display results of the preprocessing check
print("\nEnglish-French Dataset Preprocessing Check:")
print(fre_check.head())

# Detailed results for analysis
print("\nDetailed Preprocessing Checks:")
print(fre_check.apply(lambda x: f"Lowercased: {x[0]}, Punctuation Removed: {x[1]}, Tokenized: {x[2]}").head())



English-French Dataset Preprocessing Check:
0    (True, True, True)
1    (True, True, True)
2    (True, True, True)
3    (True, True, True)
4    (True, True, True)
Name: English_normalized, dtype: object

Detailed Preprocessing Checks:
0    Lowercased: True, Punctuation Removed: True, T...
1    Lowercased: True, Punctuation Removed: True, T...
2    Lowercased: True, Punctuation Removed: True, T...
3    Lowercased: True, Punctuation Removed: True, T...
4    Lowercased: True, Punctuation Removed: True, T...
Name: English_normalized, dtype: object


In [ ]:

# Updated function to normalize text with punctuation removal
def normalize_text_with_punctuation_removal(text):
    if not isinstance(text, str):  # Ensure the input is a string
        return ""
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation (only alphanumeric and spaces remain)
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    tokens = word_tokenize(text)  # Tokenize the text
    return ' '.join(tokens)  # Return the tokens as a space-separated string

# Apply the updated preprocessing function to both datasets

fre_df['English_normalized'] = fre_df['English'].apply(normalize_text_with_punctuation_removal)
fre_df['French_normalized'] = fre_df['French'].apply(normalize_text_with_punctuation_removal)

# Recheck the preprocessing for both datasets
fre_check = check_data_preprocessing(fre_df, 'English')

# Display the results of the preprocessing check
print("\nEnglish-French Dataset Preprocessing Check:")
print(fre_check.head())



English-French Dataset Preprocessing Check:
0    (True, True, True)
1    (True, True, True)
2    (True, True, True)
3    (True, True, True)
4    (True, True, True)
Name: English, dtype: object


In [ ]:
import re

def normalize_text_with_better_punctuation_removal(text):
    """
    Normalize the text by:
    1. Converting to lowercase
    2. Removing punctuation
    3. Stripping extra spaces
    """
    if not isinstance(text, str):  # Ensure input is a string
        return text
    text = text.lower()  # Convert to lowercase

    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

In [ ]:
# Apply the preprocessing function
fre_df['English_normalized'] = fre_df['English'].apply(normalize_text_with_better_punctuation_removal)
fre_df['French_normalized'] = fre_df['French'].apply(normalize_text_with_better_punctuation_removal)

# Check the preprocessing status of both datasets
fre_check = fre_df[['English', 'English_normalized', 'French', 'French_normalized']]

# Display the results
print("\nEnglish-French Dataset after preprocessing check:")
print(fre_check.head())



English-French Dataset after preprocessing check:
           English English_normalized                     French  \
0  I am a shy boy.    i am a shy boy.  Je suis un garçon timide.   
1  I am in a spot.    i am in a spot.    Je suis dans le pétrin.   
2  I am in a spot.    i am in a spot.   Je suis dans un endroit.   
3  I had to do it.    i had to do it.     Il m'a fallu le faire.   
4  I saw it on TV.    i saw it on tv.      Je l'ai vu à la télé.   

           French_normalized  
0  je suis un garçon timide.  
1    je suis dans le pétrin.  
2   je suis dans un endroit.  
3     il m'a fallu le faire.  
4      je l'ai vu à la télé.  


In [ ]:
# Check the number of rows and columns in the English-French dataset
print(f"English-French Dataset - Number of rows: {fre_df.shape[0]}")


English-French Dataset - Number of rows: 117758


In [ ]:
import tensorflow as tf
print("GPU available:", tf.config.list_physical_devices('GPU'))

GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Install swifter if not already installed
!pip install swifter

import swifter
from langdetect import detect

# Language detection function
def detect_language(text):
    try:
        return detect(text)
    except:
        return None

# Apply language detection in parallel to English-French dataset
fre_df['English_lang'] = fre_df['English_normalized'].swifter.apply(detect_language)
fre_df['French_lang'] = fre_df['French_normalized'].swifter.apply(detect_language)

# Print some rows to verify
print("\nEnglish-French Dataset after language detection:")
print(fre_df[['English_normalized', 'French_normalized', 'English_lang', 'French_lang']].head())


Pandas Apply:   0%|          | 0/117758 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/117758 [00:00<?, ?it/s]


English-French Dataset after language detection:
  English_normalized          French_normalized English_lang French_lang
0    i am a shy boy.  je suis un garçon timide.           cy          fr
1    i am in a spot.    je suis dans le pétrin.           ca          fr
2    i am in a spot.   je suis dans un endroit.           ca          fr
3    i had to do it.     il m'a fallu le faire.           en          fr
4    i saw it on tv.      je l'ai vu à la télé.           tl          fr


In [ ]:
# Define a function to remove specific punctuation from a text
def remove_punctuation(text):
    if not isinstance(text, str):  # Check if the text is a string
        return text
    return text.replace('.', '').replace('?', '').replace('!', '').replace(',', '')  # Remove specified punctuation

# Apply the function to both columns
fre_df['English_normalized'] = fre_df['English_normalized'].apply(remove_punctuation)
fre_df['French_normalized'] = fre_df['French_normalized'].apply(remove_punctuation)

# Display a preview of the updated DataFrame
print("\nUpdated Dataset without Full Stops, Question Marks, Exclamation Marks, and Commas:")
print(fre_df[['English_normalized', 'French_normalized']].head())


Updated Dataset without Full Stops, Question Marks, Exclamation Marks, and Commas:
  English_normalized         French_normalized
0     i am a shy boy  je suis un garçon timide
1     i am in a spot    je suis dans le pétrin
2     i am in a spot   je suis dans un endroit
3     i had to do it     il m'a fallu le faire
4     i saw it on tv      je l'ai vu à la télé


In [ ]:
# Filter rows where detected languages are as expected
filtered_fre_df = fre_df[(fre_df['English_lang'] == 'en') & (fre_df['French_lang'] == 'fr')]

# Display the filtered dataset
print("Filtered English-French Dataset:")
print(filtered_fre_df.head())

# Save the filtered dataset to a file (optional)
filtered_fre_df.to_csv("english_french_dataset_normalized.csv", index=False)
print(f"Filtered dataset saved to 'english_french_dataset_normalized.csv'.")


Filtered English-French Dataset:
    Unnamed: 0            English                          French  \
3         7884    I had to do it.          Il m'a fallu le faire.   
6        10675   I can do it now.    Je peux le faire maintenant.   
19       13409  5 is less than 8.   Cinq est plus petit que huit.   
21       13557  Can I be of help?      Est-ce que je peux aider ?   
22       13559  Can I go to work?  Puis-je me rendre au travail ?   

   English_normalized              French_normalized English_lang French_lang  
3      i had to do it          il m'a fallu le faire           en          fr  
6     i can do it now    je peux le faire maintenant           en          fr  
19   5 is less than 8   cinq est plus petit que huit           en          fr  
21   can i be of help      est-ce que je peux aider            en          fr  
22   can i go to work  puis-je me rendre au travail            en          fr  
Filtered dataset saved to 'english_french_dataset_normalized.csv'.


In [ ]:
from google.colab import files
files.download('english_french_dataset_normalized.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
output_path = '/content/drive/My Drive/results.csv'
filtered_fre_df.to_csv(output_path)

NameError: name 'filtered_fre_df' is not defined

In [1]:
!pip install nltk

In [2]:
# Import required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from sklearn.model_selection import train_test_split


In [3]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Define the file path to your dataset
# Ensure the file is correctly located in Google Drive, e.g., inside My Drive
file_path = '/content/drive/My Drive/english_french_dataset_normalized.csv'

# Step 3: Load the dataset using pandas
import pandas as pd

# Load the CSV file into a pandas DataFrame
try:
    data = pd.read_csv(file_path, encoding='latin-1')  # Specify 'latin-1' encoding
    print("Dataset loaded successfully!")
    print(f"Number of rows: {len(data)}")
except FileNotFoundError:
    print("File not found. Please check the file path.")

# Display the first few rows of the dataset
print(data.head())


Mounted at /content/drive
Dataset loaded successfully!
Number of rows: 105291
   Unnamed: 0            English                          French  \
0        7884    I had to do it.          Il m'a fallu le faire.   
1       10675   I can do it now.    Je peux le faire maintenant.   
2       13409  5 is less than 8.   Cinq est plus petit que huit.   
3       13557  Can I be of help?      Est-ce que je peux aider ?   
4       13559  Can I go to work?  Puis-je me rendre au travail ?   

             source                         target English_lang French_lang  
0    i had to do it          il m'a fallu le faire           en          fr  
1   i can do it now    je peux le faire maintenant           en          fr  
2  5 is less than 8   cinq est plus petit que huit           en          fr  
3  can i be of help      est-ce que je peux aider            en          fr  
4  can i go to work  puis-je me rendre au travail            en          fr  


In [4]:
# Step 1: Install required libraries
!pip install transformers torch pandas scikit-learn datasets --quiet

# Import necessary modules
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
import tensorflow as tf
print("GPU available:", tf.config.list_physical_devices('GPU'))


GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
# Display the first few rows and column names
print(data.head())  # View the data
print(data.columns)  # View the column names


   Unnamed: 0            English                          French  \
0        7884    I had to do it.          Il m'a fallu le faire.   
1       10675   I can do it now.    Je peux le faire maintenant.   
2       13409  5 is less than 8.   Cinq est plus petit que huit.   
3       13557  Can I be of help?      Est-ce que je peux aider ?   
4       13559  Can I go to work?  Puis-je me rendre au travail ?   

             source                         target English_lang French_lang  
0    i had to do it          il m'a fallu le faire           en          fr  
1   i can do it now    je peux le faire maintenant           en          fr  
2  5 is less than 8   cinq est plus petit que huit           en          fr  
3  can i be of help      est-ce que je peux aider            en          fr  
4  can i go to work  puis-je me rendre au travail            en          fr  
Index(['Unnamed: 0', 'English', 'French', 'source', 'target', 'English_lang',
       'French_lang'],
      dtype='object')


In [7]:
# Ensure dataset has 'source' (English) and 'target' (French) columns
data.rename(columns={"English_Text": "source", "French_Text": "target"}, inplace=True)

# Drop rows with missing values
data.dropna(subset=["source", "target"], inplace=True)

# Shuffle the dataset
data = data.sample(frac=1).reset_index(drop=True)

print(f"Dataset size after preprocessing: {len(data)}")


Dataset size after preprocessing: 105291


In [8]:
# Step 3: Split data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print(f"Training set size: {len(train_data)}")
print(f"Testing set size: {len(test_data)}")

# Convert data to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

Training set size: 84232
Testing set size: 21059


In [9]:
assert len(data) == len(train_data) + len(test_data), "Data split sizes do not match the original dataset size."

In [10]:
# Assuming `data` has a 'label' column, but it might be named differently
# Replace 'English_lang' or 'French_lang' with the actual column name if needed
print("Train label distribution (English_lang):")
print(train_data['English_lang'].value_counts(normalize=True))

print("Test label distribution (English_lang):")
print(test_data['English_lang'].value_counts(normalize=True))

print("Train label distribution (French_lang):")
print(train_data['French_lang'].value_counts(normalize=True))

print("Test label distribution (French_lang):")
print(test_data['French_lang'].value_counts(normalize=True))

Train label distribution (English_lang):
English_lang
en    1.0
Name: proportion, dtype: float64
Test label distribution (English_lang):
English_lang
en    1.0
Name: proportion, dtype: float64
Train label distribution (French_lang):
French_lang
fr    1.0
Name: proportion, dtype: float64
Test label distribution (French_lang):
French_lang
fr    1.0
Name: proportion, dtype: float64


In [11]:
# Assuming your labels are in either 'English_lang' or 'French_lang' columns
# Choose the appropriate column for stratification

# If you want to stratify by English language:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data['English_lang'])

# If you want to stratify by French language:
# train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data['French_lang'])

In [12]:
print("Sample from training data:")
print(train_data.head())

print("Sample from testing data:")
print(test_data.head())

Sample from training data:
       Unnamed: 0                                            English  \
29860      150113         I've lived here for more than three years.   
84714       62475                          I'm pretty proud of that.   
30442      123157                 You can smell the ocean from here.   
73642      169910  Japan is one of the greatest economic powers i...   
72653      128200               A seam on the blanket has unraveled.   

                                                  French  \
29860               Je vis ici depuis plus de trois ans.   
84714                        Je suis assez fier de cela.   
30442                  Vous pouvez sentir l'océan d'ici.   
73642  Le Japon est l'une des plus grandes puissances...   
72653        Une couture de la couverture s'est défaite.   

                                                  source  \
29860          i've lived here for more than three years   
84714                           i'm pretty proud of that   

In [13]:
print("Hugging Face train dataset size:", len(train_dataset))
print("Hugging Face test dataset size:", len(test_dataset))

Hugging Face train dataset size: 84232
Hugging Face test dataset size: 21059


In [14]:
# Install sacremoses to avoid warnings and ensure smooth tokenization
!pip install sacremoses --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 36.3 MB/s eta 0:00:00


In [15]:
# Step 4: Load pre-trained MarianMT model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Tokenize the dataset
def preprocess_function(batch):
    inputs = tokenizer(batch["source"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(batch["target"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Set PyTorch format
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Map:   0%|          | 0/84232 [00:00<?, ? examples/s]

Map:   0%|          | 0/21059 [00:00<?, ? examples/s]

In [16]:
import warnings
warnings.filterwarnings('ignore')  # To suppress warnings

In [17]:
import torch
import time

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not enabled. Please enable GPU in Runtime settings.")

# Sample computation-heavy task
def heavy_computation():
    # Simulating a large tensor operation
    tensor_size = 10000
    a = torch.randn(tensor_size, tensor_size, device=device)
    b = torch.randn(tensor_size, tensor_size, device=device)
    start_time = time.time()
    c = torch.matmul(a, b)
    torch.cuda.synchronize()  # Ensure all GPU tasks finish before measuring time
    print(f"Computation completed in {time.time() - start_time:.2f} seconds")

# Optional: Enable mixed precision for faster computation (if supported)
@torch.cuda.amp.autocast()
def heavy_computation_with_amp():
    heavy_computation()

# Execute the computation
heavy_computation()

Using device: cuda
GPU Name: Tesla T4
Computation completed in 0.67 seconds


In [18]:
from transformers import DataCollatorForSeq2Seq

# Step 5: Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
    fp16=torch.cuda.is_available(),  # Enable mixed precision if GPU is available
    report_to="none",  # Disable Weights & Biases integration
)

# Create a data collator for padding and preparing data
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Initialize Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,  # Replace tokenizer with data_collator
)

In [19]:
# Step 6: Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.073300,0.072599
2,0.067300,0.062995
3,0.064900,0.060605


TrainOutput(global_step=15795, training_loss=0.08725827693486372, metrics={'train_runtime': 3051.6278, 'train_samples_per_second': 82.807, 'train_steps_per_second': 5.176, 'total_flos': 8565981223845888.0, 'train_loss': 0.08725827693486372, 'epoch': 3.0})

In [26]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.6 MB/s eta 0:00:00


In [28]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00


In [29]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [30]:
!pip install matplotlib-venn

In [32]:
!pip install evaluate
import evaluate  # Add this line to import the 'evaluate' module

bleu_metric = evaluate.load("sacrebleu")

In [33]:
print(test_data[:5])

        Unnamed: 0                                          English  \
97504        60178                        He has not yet succeeded.   
10267        85429                     That's what friends are for.   
36018       159795  The petals floated on the surface of the water.   
69625        45833                          A wolf cannot be tamed.   
100306       79268                      Tom seems a little nervous.   

                                                 French  \
97504                         Il n'a pas encore réussi.   
10267                    Voilà à quoi servent les amis.   
36018   Les pétales flottaient sur la surface de l'eau.   
69625                  Un loup ne peut être apprivoisé.   
100306                      Tom a l'air un peu nerveux.   

                                                source  \
97504                         he has not yet succeeded   
10267                      that's what friends are for   
36018   the petals floated on the surface of

In [34]:
def translate_realtime(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    tokenized_text = tokenizer(text, return_tensors="pt", max_length=128, truncation=True, padding="max_length").to(device)

    # Use beam search to improve translation quality
    with torch.no_grad():
        translation = model.generate(
            **tokenized_text,
            num_beams=4,  # Use beam search for better translation quality
            max_length=128,
            early_stopping=True
        )

    translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)

    return translated_text


In [35]:
test_sentences = [
    "How are you today?",
    "What is your name?",
    "I love programming.",
    "It's a beautiful day."
]

for sentence in test_sentences:
    translation = translate_realtime(sentence)
    print(f"Original: {sentence}")
    print(f"Translated: {translation}")

Original: How are you today?
Translated: comment ça va aujourd'hui?
Original: What is your name?
Translated: quel est ton nom?
Original: I love programming.
Translated: j'adore la programmeur
Original: It's a beautiful day.
Translated: c'est une belle journée


In [36]:
test_sentences = [
    "Where are you from?",
    "I enjoy learning new languages.",
    "The weather is perfect today.",
    "I like to read books in my free time.",
]

for sentence in test_sentences:
    translation = translate_realtime(sentence)
    print(f"Original: {sentence}")
    print(f"Translated: {translation}")


Original: Where are you from?
Translated: d'où vous rendez-vous?
Original: I enjoy learning new languages.
Translated: j'apprécie apprendre de nouvelles langues
Original: The weather is perfect today.
Translated: le temps est parfait aujourd'hui
Original: I like to read books in my free time.
Translated: j'aime lire des livres pendant mon temps libre


In [37]:
test_data = test_data.to_dict("records")  # Convert DataFrame rows to dictionaries

In [38]:
def evaluate_model(test_data, batch_size=16):
    references = []  # Ground truth translations
    predictions = []  # Model predictions

    # Iterate through the test dataset in batches
    for i in range(0, len(test_data), batch_size):
        # Extract the current batch
        batch = test_data[i : i + batch_size]
        batch = [{"source": entry["source"], "target": entry["target"]} for entry in batch]

        # Convert batch to DataFrame
        batch_df = pd.DataFrame(batch)
        src_texts = batch_df["source"].tolist()
        tgt_texts = batch_df["target"].tolist()

        # Generate translations
        translated_texts = translation_pipeline(src_texts, max_length=256, truncation=True)
        preds = [t["translation_text"] for t in translated_texts]

        # Append predictions and references
        references.extend([[tgt] for tgt in tgt_texts])  # BLEU expects list of lists
        predictions.extend(preds)

    # Compute BLEU score
    bleu = bleu_metric.compute(predictions=predictions, references=references)

    print(f"BLEU Score: {bleu['score']:.2f}")
    return predictions, references, bleu

In [41]:
!pip install transformers
from transformers import pipeline

def evaluate_model(test_data_input, batch_size=16): # Rename input parameter to avoid conflict
    references = []  # Ground truth translations
    predictions = []  # Model predictions

    # Initialize the translation pipeline
    translation_pipeline = pipeline("translation_en_to_fr")  # Create the pipeline here

    # Iterate through the test dataset in batches
    for i in range(0, len(test_data_input), batch_size):  # Use test_data_input for iteration
        # Extract the current batch
        batch = test_data_input[i : i + batch_size]
        batch = [{"source": entry["source"], "target": entry["target"]} for entry in batch]

        # Convert batch to DataFrame
        batch_df = pd.DataFrame(batch)
        src_texts = batch_df["source"].tolist()
        tgt_texts = batch_df["target"].tolist()

        # Generate translations
        translated_texts = translation_pipeline(src_texts, max_length=256, truncation=True)
        preds = [t["translation_text"] for t in translated_texts]

        # Append predictions and references
        references.extend([[tgt] for tgt in tgt_texts])  # BLEU expects list of lists
        predictions.extend(preds)

    # Compute BLEU score
    bleu = bleu_metric.compute(predictions=predictions, references=references)

    print(f"BLEU Score: {bleu['score']:.2f}")
    return predictions, references, bleu

# Example usage with a sample dataset:
test_data = [
    {"source": "Hello, how are you?", "target": "Bonjour, comment ça va ?"},
    {"source": "I am fine, thank you.", "target": "Je vais bien, merci."},
]

predictions, references, bleu_score = evaluate_model(test_data)  # Pass the test_data to the function
print(f"Predictions: {predictions[:5]}")
print(f"References: {references[:5]}")
print(f"BLEU Score: {bleu_score['score']:.2f}")

No model was supplied, defaulted to google-t5/t5-base and revision a9723ea (https://huggingface.co/google-t5/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


BLEU Score: 19.10
Predictions: ['Bonjour, comment êtes-vous?', 'Je suis bien, je vous remercie.']
References: [['Bonjour, comment ça va ?'], ['Je vais bien, merci.']]
BLEU Score: 19.10


In [42]:
# Define the local save path
local_model_path = "./trained_model"

# Save the trained model and tokenizer locally
model.save_pretrained(local_model_path)
tokenizer.save_pretrained(local_model_path)

print(f"Model and tokenizer saved locally at {local_model_path}")

Model and tokenizer saved locally at ./trained_model


In [46]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define the Google Drive save path
drive_model_path = "/content/drive/My Drive/trained_model"

# Copy the locally saved model to Google Drive
shutil.copytree(local_model_path, drive_model_path)

print(f"Model and tokenizer saved to Google Drive at {drive_model_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model and tokenizer saved to Google Drive at /content/drive/My Drive/trained_model


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
!pip install sacremoses

In [22]:
model_path = "/content/drive/MyDrive/trained_model"

In [23]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Define the model path (from local or Google Drive)
model_path = "/content/drive/MyDrive/trained_model"  # Update the path

# Load the model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [24]:
def translate_realtime(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    tokenized_text = tokenizer(text, return_tensors="pt", max_length=128, truncation=True, padding="max_length").to(device)

    # Use beam search to improve translation quality
    with torch.no_grad():
        translation = model.generate(
            **tokenized_text,
            num_beams=4,  # Use beam search for better translation quality
            max_length=128,
            early_stopping=True
        )

    translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)

    return translated_text


In [25]:
test_sentences = [
    "How are you today?",
    "What is your name?",
    "I love programming.",
    "It's a beautiful day."
]

for sentence in test_sentences:
    translation = translate_realtime(sentence)
    print(f"Original: {sentence}")
    print(f"Translated: {translation}")

Original: How are you today?
Translated: comment ça va aujourd'hui?
Original: What is your name?
Translated: quel est ton nom?
Original: I love programming.
Translated: j'adore la programmeur
Original: It's a beautiful day.
Translated: c'est une belle journée


In [47]:
test_sentences = [
    "Where are you from?",
    "I enjoy learning new languages.",
    "The weather is perfect today.",
    "I like to read books in my free time.",
]

for sentence in test_sentences:
    translation = translate_realtime(sentence)
    print(f"Original: {sentence}")
    print(f"Translated: {translation}")


Original: Where are you from?
Translated: d'où vous rendez-vous?
Original: I enjoy learning new languages.
Translated: j'apprécie apprendre de nouvelles langues
Original: The weather is perfect today.
Translated: le temps est parfait aujourd'hui
Original: I like to read books in my free time.
Translated: j'aime lire des livres pendant mon temps libre


In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
model_path = "/content/drive/MyDrive/trained_model"

In [50]:
!pip install sacremoses

In [51]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Define the model path (from local or Google Drive)
model_path = "/content/drive/MyDrive/trained_model"  # Update the path

# Load the model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [52]:
!pip install matplotlib-venn

In [53]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [54]:
!pip install evaluate

In [55]:
!pip install sacrebleu

In [56]:
bleu_metric = evaluate.load("sacrebleu")

In [57]:
print(test_data[:5])

[{'source': 'Hello, how are you?', 'target': 'Bonjour, comment ça va ?'}, {'source': 'I am fine, thank you.', 'target': 'Je vais bien, merci.'}]


In [59]:
def evaluate_model(test_data, batch_size=16):
    references = []  # Ground truth translations
    predictions = []  # Model predictions

    # Iterate through the test dataset in batches
    for i in range(0, len(test_data), batch_size):
        # Extract the current batch
        batch = test_data[i : i + batch_size]
        batch = [{"source": entry["source"], "target": entry["target"]} for entry in batch]

        # Convert batch to DataFrame
        batch_df = pd.DataFrame(batch)
        src_texts = batch_df["source"].tolist()
        tgt_texts = batch_df["target"].tolist()

        # Generate translations
        translated_texts = translation_pipeline(src_texts, max_length=256, truncation=True)
        preds = [t["translation_text"] for t in translated_texts]

        # Append predictions and references
        references.extend([[tgt] for tgt in tgt_texts])  # BLEU expects list of lists
        predictions.extend(preds)

    # Compute BLEU score
    bleu = bleu_metric.compute(predictions=predictions, references=references)

    print(f"BLEU Score: {bleu['score']:.2f}")
    return predictions, references, bleu

In [61]:
!pip install transformers
from transformers import pipeline

def evaluate_model(test_data_input, batch_size=16): # Rename input parameter to avoid conflict
    references = []  # Ground truth translations
    predictions = []  # Model predictions

    # Initialize the translation pipeline
    translation_pipeline = pipeline("translation_en_to_fr")  # Create the pipeline here

    # Iterate through the test dataset in batches
    for i in range(0, len(test_data_input), batch_size):  # Use test_data_input for iteration
        # Extract the current batch
        batch = test_data_input[i : i + batch_size]
        batch = [{"source": entry["source"], "target": entry["target"]} for entry in batch]

        # Convert batch to DataFrame
        batch_df = pd.DataFrame(batch)
        src_texts = batch_df["source"].tolist()
        tgt_texts = batch_df["target"].tolist()

        # Generate translations
        translated_texts = translation_pipeline(src_texts, max_length=256, truncation=True)
        preds = [t["translation_text"] for t in translated_texts]

        # Append predictions and references
        references.extend([[tgt] for tgt in tgt_texts])  # BLEU expects list of lists
        predictions.extend(preds)

    # Compute BLEU score
    bleu = bleu_metric.compute(predictions=predictions, references=references)

    print(f"BLEU Score: {bleu['score']:.2f}")
    return predictions, references, bleu

# Example usage with a sample dataset:
test_data = [
    {"source": "Hello, how are you?", "target": "Bonjour, comment ça va ?"},
    {"source": "I am fine, thank you.", "target": "Je vais bien, merci."},
]

predictions, references, bleu_score = evaluate_model(test_data)  # Pass the test_data to the function
print(f"Predictions: {predictions[:5]}")
print(f"References: {references[:5]}")
print(f"BLEU Score: {bleu_score['score']:.2f}")

No model was supplied, defaulted to google-t5/t5-base and revision a9723ea (https://huggingface.co/google-t5/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


BLEU Score: 19.10
Predictions: ['Bonjour, comment êtes-vous?', 'Je suis bien, je vous remercie.']
References: [['Bonjour, comment ça va ?'], ['Je vais bien, merci.']]
BLEU Score: 19.10


In [62]:
def translate_realtime(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    tokenized_text = tokenizer(text, return_tensors="pt", max_length=128, truncation=True, padding="max_length").to(device)

    # Use beam search to improve translation quality
    with torch.no_grad():
        translation = model.generate(
            **tokenized_text,
            num_beams=4,  # Use beam search for better translation quality
            max_length=128,
            early_stopping=True
        )

    translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)

    return translated_text


In [63]:
test_sentences = [
    "How are you today?",
    "What is your name?",
    "I love programming.",
    "It's a beautiful day."
]

for sentence in test_sentences:
    translation = translate_realtime(sentence)
    print(f"Original: {sentence}")
    print(f"Translated: {translation}")

Original: How are you today?
Translated: comment ça va aujourd'hui?
Original: What is your name?
Translated: quel est ton nom?
Original: I love programming.
Translated: j'adore la programmeur
Original: It's a beautiful day.
Translated: c'est une belle journée


In [64]:
# Define the local save path
local_model_path = "./trained_model"

# Save the trained model and tokenizer locally
model.save_pretrained(local_model_path)
tokenizer.save_pretrained(local_model_path)

print(f"Model and tokenizer saved locally at {local_model_path}")

Model and tokenizer saved locally at ./trained_model


In [66]:
pip install streamlit transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.4 MB/s eta 0:00:00


In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
import warnings
warnings.filterwarnings('ignore')  # To suppress warnings

In [69]:
def translate_realtime(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    tokenized_text = tokenizer(text, return_tensors="pt", max_length=128, truncation=True, padding="max_length").to(device)

    # Use beam search to improve translation quality
    with torch.no_grad():
        translation = model.generate(
            **tokenized_text,
            num_beams=4,  # Use beam search for better translation quality
            max_length=128,
            early_stopping=True
        )

    translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)

    return translated_text


In [70]:
test_sentences = [
    "How are you today?",
    "What is your name?",
    "I love programming.",
    "It's a beautiful day."
]

for sentence in test_sentences:
    translation = translate_realtime(sentence)
    print(f"Original: {sentence}")
    print(f"Translated: {translation}")

Original: How are you today?
Translated: comment ça va aujourd'hui?
Original: What is your name?
Translated: quel est ton nom?
Original: I love programming.
Translated: j'adore la programmeur
Original: It's a beautiful day.
Translated: c'est une belle journée


In [71]:
test_sentences = [
    "Where are you from?",
    "I enjoy learning new languages.",
    "The weather is perfect today.",
    "I like to read books in my free time.",
    "I recently started a new project at work. It's very exciting because I get to work with a great team. We're focusing on innovative ideas and trying to make a difference in the industry. I hope this project will lead to many new opportunities for everyone involved. In the future, I want to keep growing professionally and contribute to exciting projects like this one."
]

for sentence in test_sentences:
    translation = translate_realtime(sentence)
    print(f"Original: {sentence}")
    print(f"Translated: {translation}")


Original: Where are you from?
Translated: d'où vous rendez-vous?
Original: I enjoy learning new languages.
Translated: j'apprécie apprendre de nouvelles langues
Original: The weather is perfect today.
Translated: le temps est parfait aujourd'hui
Original: I like to read books in my free time.
Translated: j'aime lire des livres pendant mon temps libre
Original: I recently started a new project at work. It's very exciting because I get to work with a great team. We're focusing on innovative ideas and trying to make a difference in the industry. I hope this project will lead to many new opportunities for everyone involved. In the future, I want to keep growing professionally and contribute to exciting projects like this one.
Translated: j'ai récemment commencé un nouveau projet au travail c'est très passionnant parce que j'arrive à travailler avec une grande équipe c'est nous sommes concentrés sur des idées innovantes et que nous essayons de faire une différence dans l'industrie j'espère 

In [72]:
def translate_realtime(text):
    # Move model to the correct device (GPU if available, else CPU)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    # Tokenize the input text with padding to handle all sequences correctly
    tokenized_text = tokenizer(
        text,
        return_tensors="pt",
        max_length=128,
        truncation=True,
        padding="longest"  # Use longest padding to handle varying sentence lengths
    ).to(device)

    # Perform translation using beam search for better results
    with torch.no_grad():
        translation = model.generate(
            **tokenized_text,
            num_beams=4,  # Use beam search for better translations
            max_length=128,
            early_stopping=True  # Stop early when the best translation is found
        )

    # Decode the translated tokens back to text
    translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)

    return translated_text


In [73]:
!pip install optuna transformers torch datasets scikit-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.0 MB/s eta 0:00:00


In [74]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Replace with your model and tokenizer
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/source.spm',
 './trained_model/target.spm',
 './trained_model/added_tokens.json')

In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
model_path = "/content/drive/MyDrive/trained_model"

In [77]:
!pip install streamlit transformers PyPDF2 fpdf


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.8 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=b6db4b546efc3fcc388b09b4a9b4a2f0c669171ee5f785a89c4166624f9a930e
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [78]:
!pip install transformers

In [79]:
!pip install --upgrade torch

In [80]:
!pip install --upgrade torch torchvision torchaudio

In [ ]:
# Restart runtime after upgrade to apply changes
import os
os.kill(os.getpid(), 9)

In [81]:
!pip install sacremoses

In [82]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Define the model path (from local or Google Drive)
model_path = "/content/drive/MyDrive/trained_model"  # Update the path

# Load the model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [83]:
!ls

app.py	drive  results	sample_data  trained_model


In [84]:
input_text = "Hello, how are you?"

# Tokenize the input text
inputs = tokenizer.encode(input_text, return_tensors="pt")

# Generate the translation
outputs = model.generate(inputs, max_length=50, num_beams=4, early_stopping=True)

# Decode the generated output
translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Translated Text: ", translation)

Translated Text:  allô ça va ?


In [85]:
model.save_pretrained("/content/trained_model")
tokenizer.save_pretrained("/content/trained_model")

# If you want to download the model:
from google.colab import files
!zip -r /content/trained_model.zip /content/trained_model
files.download('/content/trained_model.zip')

  adding: content/trained_model/ (stored 0%)
  adding: content/trained_model/vocab.json (deflated 70%)
  adding: content/trained_model/source.spm (deflated 49%)
  adding: content/trained_model/generation_config.json (deflated 43%)
  adding: content/trained_model/config.json (deflated 61%)
  adding: content/trained_model/model.safetensors (deflated 7%)
  adding: content/trained_model/target.spm (deflated 50%)
  adding: content/trained_model/tokenizer_config.json (deflated 68%)
  adding: content/trained_model/special_tokens_map.json (deflated 73%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [86]:
!pip install pyngrok

In [87]:
!nohup streamlit run app.py &>/dev/null &

In [88]:
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2qFpS8rfO1fs20xEpjB3hYrin2i_FaEULxgMaKBHoAhJWNcp")

In [89]:
!pip install --upgrade pyngrok

In [90]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 85.4 MB/s eta 0:00:00


In [91]:
!pip install language_tool_python

In [110]:
!pip install onnxruntime-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.4 MB/s eta 0:00:00


In [111]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 61.3 MB/s eta 0:00:00


In [112]:
import torch

# Clear GPU memory
def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

# Call the function
clear_gpu_memory()

print("GPU memory cleared!")


GPU memory cleared!


In [1]:
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2qFpS8rfO1fs20xEpjB3hYrin2i_FaEULxgMaKBHoAhJWNcp")

# Open a tunnel on port 8501 (Streamlit default port)
public_url = ngrok.connect(8501)
print('Streamlit App is live at:', public_url)

Streamlit App is live at: NgrokTunnel: "https://5c1a-34-87-123-59.ngrok-free.app" -> "http://localhost:8501"


In [2]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.123.59:8501

2024-12-16 14:29:26.756809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-16 14:29:26.781638: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-16 14:29:26.789272: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-16 14:29:26.807234: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To 

Streamlit App is live at: NgrokTunnel: "https://ba0e-34-132-57-243.ngrok-free.app" -> "http://localhost:8501"


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
